In [2]:
%pip install pandas
%pip install nltk
%pip install scikit-learn
%pip install sklearn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post4.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\Hp\AppData\Local\Temp\pip-install-2nzp3a93\sklearn_40ccd8695a3c4d1a88f51e041a5ce9aa\setup.py", line 10, in <module>
          LONG_DESCRIPTION = f.read()
        File "c:\Python\lib\encodings\cp1252.py", line 23, in decode
          return codecs.charmap_decode(input,self.errors,decoding_table)[0]
      UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 7: character maps to <undefined>
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned abo

In [3]:
import pandas as pd
import nltk
import numpy as np
import math
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Initially Viewing the Data

In [4]:
df = pd.read_csv("emails.csv")
df.head()
print(df.shape)

(5172, 3002)


## Preprocessing Pipeline

In [5]:
# Import the list of stop words from the 'stopwords' module
stopwordList = stopwords.words('english')

# Identify the columns in the DataFrame that have names matching any of the stop words
# Convert the stop word list and DataFrame columns to sets, find the intersection, and convert back to a list
columnsToDrop = list(set(stopwordList) & set(df.columns))

# Drop the identified columns from the DataFrame
# 'axis=1' specifies that the operation should be performed on columns (as opposed to rows)
df = df.drop(columnsToDrop, axis=1)

# Print the first few rows of the resulting DataFrame
df.head()

,Email No.,ect,hou,enron,com,gas,deal,meter,hpl,please,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,24,27,1,3,1,0,0,0,2,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,1,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,22,10,0,0,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,17,9,0,0,2,0,3,0,1,...,0,0,0,0,0,0,0,1,0,0


In [6]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
print("Train shape: ", train_df.shape)
print("Test shape: ", test_df.shape)
print("Train spam ratio: ", train_df['spam'].mean())
print("Test spam ratio: ", test_df['spam'].mean())
print(train_df.head())
print(test_df.head())

Train shape:  (4137, 2868)
Test shape:  (1035, 2868)
Train spam ratio:  0.019579405366207395
Test spam ratio:  0.010628019323671498
       Email No.  ect  hou  enron  com  gas  deal  meter  hpl  please  ...   
3164  Email 3165    2    2      0    3    0     0      0    0       0  ...  \
2067  Email 2068    1    0      0    0    0     0      0    3       0  ...   
4717  Email 4718    4    4      0    6    0     0      0    0       0  ...   
2505  Email 2506    1    0      0    0    1     0      1    0       0  ...   
2268  Email 2269    3    1      0    1    0     0      0    0       0  ...   

      connevey  jay  valued  lay  infrastructure  military  allowing  ff  dry   
3164         0    0       0    0               0         0         0   1    0  \
2067         0    0       0    0               0         0         0   0    0   
4717         0    0       0    0               0         0         0   3    0   
2505         0    0       0    0               0         0         0   0   

In [7]:
new_train_df = train_df.drop(columns=["Email No.", "Prediction"])
new_test_df = test_df.drop(columns=["Email No.", "Prediction"])
doc_freq_train = new_train_df.astype(bool).sum(axis=0)
print(doc_freq_train.sort_values(ascending=False).head(10))
print(len(doc_freq_train))
doc_freq_test = new_test_df.astype(bool).sum(axis=0)
print(doc_freq_test.sort_values(ascending=False).head(10))
print(len(doc_freq_test))

ect    4137
j      4137
u      4137
b      4137
ct     4137
e      4137
c      4137
r      4099
n      4093
l      4072
dtype: int64
2866
ect    1035
j      1035
c      1035
ct     1035
u      1035
e      1035
b      1035
r      1025
n      1025
l      1020
dtype: int64
2866


In [8]:
number_doc_train = new_train_df.shape[0]
# calculating idf for each word
idf_train = np.log(number_doc_train / doc_freq_train)
idf_test = np.log(number_doc_train / doc_freq_test)
print(idf_train)
print(idf_test)

ect               0.000000
hou               0.938780
enron             1.241825
com               0.909545
gas               1.447342
                    ...   
infrastructure    5.929831
military          5.762777
allowing          5.555137
ff                1.045653
dry               5.108850
Length: 2866, dtype: float64
ect               1.385569
hou               2.351375
enron             2.682279
com               2.277993
gas               2.826468
                    ...   
infrastructure    7.634579
military          6.941432
allowing          7.229114
ff                2.341274
dry               6.381816
Length: 2866, dtype: float64


In [9]:
# calculating tf-idf for each word
tf_idf_train = new_train_df * idf_train
tf_idf_test = new_test_df * idf_test
print(tf_idf_train)
print(tf_idf_test)

      ect      hou     enron       com       gas      deal     meter   
3164  0.0  1.87756  0.000000  2.728636  0.000000  0.000000  0.000000  \
2067  0.0  0.00000  0.000000  0.000000  0.000000  0.000000  0.000000   
4717  0.0  3.75512  0.000000  5.457272  0.000000  0.000000  0.000000   
2505  0.0  0.00000  0.000000  0.000000  1.447342  0.000000  1.796849   
2268  0.0  0.93878  0.000000  0.909545  0.000000  0.000000  0.000000   
...   ...      ...       ...       ...       ...       ...       ...   
4426  0.0  0.00000  2.483649  1.819091  0.000000  0.000000  0.000000   
466   0.0  0.00000  2.483649  0.000000  1.447342  1.468111  0.000000   
3092  0.0  0.00000  2.483649  0.000000  1.447342  0.000000  0.000000   
3772  0.0  0.00000  0.000000  2.728636  0.000000  0.000000  0.000000   
860   0.0  0.00000  0.000000  0.000000  0.000000  0.000000  0.000000   

           hpl    please    e  ...  enhancements  connevey  jay  valued   
3164  0.000000  0.000000  0.0  ...           0.0       0.0  

In [10]:
def vectorMagnitude(vector):
    Sum=0
    for value in vector:
        Sum+=math.pow(value,2)
    return math.sqrt(Sum)

In [ ]:
tf_idf_train_array=tf_idf_train.to_numpy()
print(len(tf_idf_train_array[0]))
tf_idf_test_array=tf_idf_test.to_numpy()
print(len(tf_idf_test_array))

In [17]:
def KNNClassifier(k):
    cosineSim=[]
    index=0
    newVector=tf_idf_test_array[0]*tf_idf_train_array[0]
    newVector=np.nan_to_num(newVector.data)
    for testVector,testIndex in zip(tf_idf_test_array,test_df.index):
        cosineSim.append([])
        testVector=np.nan_to_num(testVector)
        for trainVector, trainIndex in zip(tf_idf_train_array, train_df.index):
            trainVector=np.nan_to_num(trainVector)
            dotProduct=testVector*trainVector
            sumOfDotProduct=sum(dotProduct)
            cosineSim[index].append((sumOfDotProduct/(vectorMagnitude(testVector)*vectorMagnitude(trainVector)),train_df.loc[trainIndex, 'Prediction']))
        cosineSim[index]=sorted(cosineSim[index],reverse=True)
        cosineSim[index]=cosineSim[index][:k]
        cosineSim[index].append(test_df.loc[testIndex, 'Prediction'])
        print(cosineSim[index])
        index+=1
    return cosineSim

In [23]:
def performanceMetric():
    for k in range(2,11):
        cosineSim=KNNClassifier(k)
        correct=0
        spamCount=0
        nonSpamCount=0
        for values in range(len(cosineSim)):
            for row in range(len(cosineSim[values])-1):
                if cosineSim[values][row][1]==1:
                    spamCount+=1
                else:
                    nonSpamCount+=1
            if spamCount > nonSpamCount and cosineSim[values][len(cosineSim[values])-1][1]==1:
                correct+=1
            elif spamCount < nonSpamCount and cosineSim[values][len(cosineSim[values])-1][1]==0:
                correct+=1
            else:
                if cosineSim[values][len(cosineSim[values])-1][1] == 1:
                    correct += 1
            spamCount=0
            nonSpamCount=0
        print("k=",k)
        print("Accuracy=",correct/len(test_df))
        


In [24]:
performanceMetric()

C:\Users\Hp\AppData\Local\Temp\ipykernel_17340\770724222.py:13: RuntimeWarning: invalid value encountered in scalar divide
  cosineSim[index].append((sumOfDotProduct/(vectorMagnitude(testVector)*vectorMagnitude(trainVector)),train_df.loc[trainIndex, 'Prediction']))


[(0.47111341244343513, 0), (0.3888507227205867, 0), 0]
[(0.4051717028455861, 0), (0.4032659216568999, 0), 0]


KeyboardInterrupt: 